# Spectrogram Emotion Prototype Learning

*Code fully written by Daniel Peña Porras*.



---


*    #### Training and evaluation of **CONVOLUTIONAL AUTOENCODER** with traditional and gradient-based loss functions to ensure preservation of highly detailed parts of the spectrograms.


---


In [ ]:
'''
@author: Daniel Peña Porras.
'''

'\n@author: Daniel Peña Porras.\n'

##### Dependencies

In [ ]:
! pip install librosa
! pip install mir_eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 233.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.8/102.8 kB 18.0 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

2025-06-19 07:48:29.871247: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [ ]:
import os

import librosa
import librosa.display as dsp

import mir_eval

from IPython.display import Audio

import matplotlib.pyplot as plt
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn import preprocessing

from skimage.transform import resize
from scipy.ndimage import gaussian_laplace

from keras.callbacks import TensorBoard

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import LeakyReLU, BatchNormalization, ActivityRegularization#, Sigmoid
import os
from keras import backend as K
from tensorflow.keras.utils import plot_model, model_to_dot
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, Callback

from skimage.transform import resize
from keras.callbacks import Callback
from keras.applications import VGG19, EfficientNetB0, ResNet50V2, DenseNet121
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Lambda, Concatenate
from keras.optimizers import Adam
from keras.losses import MeanSquaredError

from scipy.ndimage import gaussian_laplace

from scipy.io import wavfile

from keras.constraints import NonNeg,UnitNorm
from keras.initializers import Constant

In [ ]:
os.environ["KERAS_BACKEND"] = "tensorflow"

##### Hyperparameters

In [ ]:
"""
DATA PARAMETERS
"""
#prepocess_window_size_ms = 100
#preprocess_window_overlapping_ms = 20
test_prctg = 0.3

dataset = 'EmoMatch' # or 'EmoDB' 'RAVDESS'

dtype = 'float32'
n_frames_cnn = None #128 established after data loading
n_freq_cnn = None #20 established after data loading
n_color_channels = 1

#same parameters as in preprocessing
n_fft = 1024            # length of the FFT window
window = 'hamming'      # window function
win_length = 512        # window length
hop_length = 256        # number of samples between successive frames
n_mels = 128

In [ ]:
"""
ENCODER HYPERPARAMETERS
"""
n_filters_enc = [64, 32, 16, 8, 4]
filters_size_cnn_enc = [(3, 3), (3, 3), (3, 3), (3, 3), (3, 3)]
strides_enc = [2, 1, 1, 1, 1]
paddings_enc = ['same', 'same', 'same', 'same', 'same']
kernel_initializer = 'he_uniform'
use_batch_norm = False
use_l1_reg = False
conv_activation = 'relu'
encoded_activation = 'relu'
encoder_name = 'encoder'

In [ ]:
"""
DECODER HYPERPARAMETERS
  kernel_initializer, use_batch_norm and use_l1_reg are the same as in the encoder
"""
n_filters_dec = [4, 8, 16, 32, 64]
filters_size_cnn_dec = [(3, 3), (3, 3), (3, 3), (3, 3), (3, 3), (3, 3)]
strides_dec = [1, 1, 1, 1, 2]
paddings_dec = ['same', 'same', 'same', 'same', 'same', 'same']
deconv_activation = 'relu'
decoded_activation = 'sigmoid'
decoder_name = 'decoder'

In [ ]:
"""
TRAINING HYPERPARAMETERS
"""

autoencoder_epochs = 100
autoencoder_batch_size = 128
autoencoder_learning_rate = 0.001


lambdaR = 1
reconstruction_loss = keras.losses.MeanSquaredError()
recons_loss_name = 'keras.losses.MeanSquaredError()'

reconstruct_with_HFENN = True
if reconstruct_with_HFENN:
    lambdaHFENN = 0.0001
    lambdaR = 1

reconstruction_patience_epochs = 50
reconstruction_convergence = 0

ratioReconstructionDisplay = 10

In [ ]:
if dataset == 'RAVDESS':
    sr = 48000
    n_speakers = 24
    complete_speakers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                         14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
    n_phonemes = 44
    n_classes = 8
    class_names = [
        'neutral',
        'calm',
        'happy',
        'sad',
        'angry',
        'fearful',
        'disgust',
        'surprised'
    ]
    get_prototype_emotion = {
        0: 'neutral',
        1: 'calm',
        2: 'joy',
        3: 'sadness',
        4: 'anger',
        5: 'fear',
        6: 'disgust',
        7: 'surprise'
    }
    male_speakers = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23]
    female_speakers = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]

elif dataset == 'EmoMatch':
    sr = 48000
    # n_speakers = 50
    speakers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 20, 22, 23,
                25, 29, 31, 33, 35, 36, 37, 40, 41, 43, 45, 46, 49]
    n_speakers = len(n_speakers)
    n_phonemes = 24
    n_classes = 7
    class_names = [
        'angry',
        'disgust',
        'fearful',
        'happy',
        'neutral',
        'sad',
        'surprised'
    ]
    get_prototype_emotion = {
        0: 'anger',
        1: 'disgust',
        2: 'fear',
        3: 'joy',
        4: 'neutral',
        5: 'sadness',
        6: 'surprise'
    }
    male_speakers = [1, 2, 5, 9, 12, 13, 22, 23, 31, 35, 36, 37, 43]
    female_speakers = [3, 4, 6, 7, 8, 10, 11, 20, 25, 29, 33, 40, 41, 45, 46, 49]

elif dataset == 'EmoDB':
    sr = 16000
    n_speakers = 10
    speakers = [3, 8, 9, 10, 11, 12, 13, 14, 15, 16]
    n_phonemes = 45
    n_classes = 7
    class_names = [
        'angry',
        'bored',
        'disgust',
        'fearful',
        'happy',
        'neutral',
        'sad'
    ]
    get_prototype_emotion = {
        0: 'angry',
        1: 'bored',
        2: 'disgust',
        3: 'fearful',
        4: 'happy',
        5: 'neutral',
        6: 'sad'
    }
    male_speakers = [3, 10, 11, 12, 15]
    female_speakers = [8, 9, 13, 14, 16]

n_prototypes = n_phonemes * n_classes

fmax = sr/2
fmin = 0  # default value if not specified

##### Models and losses

In [ ]:
def create_encoder(input_img,
                    n_filters_enc,
                    filters_size_cnn_enc,
                    strides_enc,
                    paddings_enc,
                    kernel_initializer,
                    use_batch_norm,
                    use_l1_reg,
                    conv_activation,
                    encoded_activation,
                    encoder_name):

  x = K.expand_dims(input_img,-1)
  l1 = layers.Conv2D(n_filters_enc[0], filters_size_cnn_enc[0], padding=paddings_enc[0], strides=strides_enc[0], kernel_initializer=kernel_initializer, activation=conv_activation, name='conv1')(x)
  if use_l1_reg:
    l1 = ActivityRegularization(l1=10e-10)(l1)
  if use_batch_norm:
    l1 = BatchNormalization()(l1)
  l2 = layers.Conv2D(n_filters_enc[1], filters_size_cnn_enc[1], padding=paddings_enc[1], strides=strides_enc[1], kernel_initializer=kernel_initializer, activation=conv_activation, name='conv2')(l1)
  if use_l1_reg:
    l2 = ActivityRegularization(l1=10e-10)(l2)
  if use_batch_norm:
    l2 = BatchNormalization()(l2)
  l3 = layers.Conv2D(n_filters_enc[2], filters_size_cnn_enc[2], padding=paddings_enc[2], strides=strides_enc[2], kernel_initializer=kernel_initializer, activation=conv_activation, name='conv3')(l2)
  if use_l1_reg:
    l3 = ActivityRegularization(l1=10e-10)(l3)
  if use_batch_norm:
    l3 = BatchNormalization()(l3)
  l4 = layers.Conv2D(n_filters_enc[3], filters_size_cnn_enc[3], padding=paddings_enc[3], strides=strides_enc[3], kernel_initializer=kernel_initializer, activation=conv_activation, name='conv4')(l3)
  if use_l1_reg:
    l4 = ActivityRegularization(l1=10e-10)(l4)
  if use_batch_norm:
    l4 = BatchNormalization()(l4)
  encoded = layers.Conv2D(n_filters_enc[4], filters_size_cnn_enc[4], padding=paddings_enc[4], strides=strides_enc[4], kernel_initializer=kernel_initializer, activation=encoded_activation, name='encoded')(l4)
  if use_l1_reg:
    encoded = ActivityRegularization(l1=10e-10)(encoded)
  if use_batch_norm:
    encoded = BatchNormalization()(encoded)

  return Model(input_img, encoded, name=encoder_name)

In [ ]:
def create_decoder(decoder_input,
                    n_color_channels,
                    n_filters_dec,
                    filters_size_cnn_dec,
                    strides_dec,
                    paddings_dec,
                    kernel_initializer,
                    use_batch_norm,
                    use_l1_reg,
                    deconv_activation,
                    decoded_activation,
                    decoder_name):

  l1 = layers.Conv2DTranspose(n_filters_dec[0], filters_size_cnn_dec[0], padding=paddings_dec[0], strides=strides_dec[0], kernel_initializer=kernel_initializer, activation=deconv_activation, name='deconv1')(decoder_input)
  if use_l1_reg:
    l1 = ActivityRegularization(l1=10e-10)(l1)
  if use_batch_norm:
    l1 = BatchNormalization()(l1)
  l2 = layers.Conv2DTranspose(n_filters_dec[1], filters_size_cnn_dec[1], padding=paddings_dec[1], strides=strides_dec[1], kernel_initializer=kernel_initializer, activation=deconv_activation, name='deconv2')(l1)
  if use_l1_reg:
    l2 = ActivityRegularization(l1=10e-10)(l2)
  if use_batch_norm:
    l2 = BatchNormalization()(l2)
  l3 = layers.Conv2DTranspose(n_filters_dec[2], filters_size_cnn_dec[2], padding=paddings_dec[2], strides=strides_dec[2], kernel_initializer=kernel_initializer, activation=deconv_activation, name='deconv3')(l2)
  if use_l1_reg:
    l3 = ActivityRegularization(l1=10e-10)(l3)
  if use_batch_norm:
    l3 = BatchNormalization()(l3)
  l4 = layers.Conv2DTranspose(n_filters_dec[3], filters_size_cnn_dec[3], padding=paddings_dec[3], strides=strides_dec[3], kernel_initializer=kernel_initializer, activation=deconv_activation, name='deconv4')(l3)
  if use_l1_reg:
    l4 = ActivityRegularization(l1=10e-10)(l4)
  if use_batch_norm:
    l4 = BatchNormalization()(l4)
  l5 = layers.Conv2DTranspose(n_filters_dec[4], filters_size_cnn_dec[4], padding=paddings_dec[4], strides=strides_dec[4], kernel_initializer=kernel_initializer, activation=deconv_activation, name='deconv5')(l4)
  if use_l1_reg:
    l5 = ActivityRegularization(l1=10e-10)(l5)
  if use_batch_norm:
    l5 = BatchNormalization()(l5)

  y = layers.Conv2D(n_color_channels, filters_size_cnn_dec[5], padding=paddings_dec[5], activation=decoded_activation)(l5)
  decoded = K.squeeze(y, axis=-1)

  return Model(decoder_input, decoded, name=decoder_name)

In [ ]:
def hfenn(y_true, y_pred):
    """
    Computes the HFENN (High-Frequency Error Norm) loss between y_true and y_pred.

    Arguments:
    y_true -- Ground truth tensor (batch_size, height, width, channels)
    y_pred -- Predicted tensor (batch_size, height, width, channels)

    Returns:
    hfenn_loss -- HFENN loss
    """

    y_true = K.expand_dims(y_true,-1)
    y_pred = K.expand_dims(y_pred,-1)

    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)

    # Compute gradients of y_true and y_pred
    grad_y_true = tf.image.sobel_edges(y_true)
    grad_y_pred = tf.image.sobel_edges(y_pred)

    # Compute HFENN loss
    hfenn_loss = K.mean(K.abs(grad_y_true - grad_y_pred))

    return hfenn_loss

## **Training loop**

In [ ]:
#for speaker_id in range(1, n_speakers + 1):
for speaker in speakers:
    speaker = f"{speaker_id:02d}"  # format as '01', '02', ..., '50'
    print(f"\n\n🔁 Training Autoencoder for Speaker {speaker}...")

    # Define paths
    data_path = f'/home/jovyan/Daniel/Exp_{dataset}/datasets/preprocessed/{dataset}100ms32LabeledSpe{speaker}.npz'
    store_AE_path = f'/home/jovyan/Daniel/Exp_{dataset}/autoencoders_training/models/autoencoder_Speaker{speaker}.keras'
    store_encoder_path = f'/home/jovyan/Daniel/Exp_{dataset}/autoencoders_training/models/encoder_Speaker{speaker}.keras'
    store_decoder_path = f'/home/jovyan/Daniel/Exp_{dataset}/autoencoders_training/models/decoder_Speaker{speaker}.keras'

    # Load data
    data = np.load(data_path)
    x_train_100ms = data['x_train']
    y_train_100ms = data['y_emo']

    x_train, x_test, y_train, y_test = train_test_split(
        x_train_100ms, y_train_100ms, test_size=test_prctg, random_state=42)

    # Normalization
    min_norm = np.min(x_train)
    max_norm = np.max(x_train)

    x_train_norm = (x_train - min_norm) / (max_norm - min_norm)
    x_test_norm = (x_test - min_norm) / (max_norm - min_norm)

    # Define model
    input_img = Input(shape=(x_train.shape[1], x_train.shape[2]), dtype=x_train.dtype, name='input')
    encoder = create_encoder(input_img, n_filters_enc, filters_size_cnn_enc, strides_enc, paddings_enc,
                             kernel_initializer, use_batch_norm, use_l1_reg, conv_activation,
                             encoded_activation, encoder_name)
    encoded = encoder(input_img)

    decoder_input = Input(shape=encoded.shape[1:], dtype=x_train.dtype, name='decoder_input')
    decoder = create_decoder(decoder_input, n_color_channels, n_filters_dec, filters_size_cnn_dec,
                             strides_dec, paddings_dec, kernel_initializer, use_batch_norm,
                             use_l1_reg, deconv_activation, decoded_activation, decoder_name)
    decoded = decoder(encoded)

    autoencoder = Model(input_img, decoded, name='autoencoder')

    # Training
    optimizer = keras.optimizers.Adam(learning_rate=autoencoder_learning_rate)
    train_MAE_metric = keras.metrics.MeanAbsoluteError()
    val_MAE_metric = keras.metrics.MeanAbsoluteError()

    train_dataset = tf.data.Dataset.from_tensor_slices((x_train_norm, y_train)).shuffle(512).batch(autoencoder_batch_size)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test_norm, y_test)).batch(autoencoder_batch_size)

    train_losses = []
    train_MAEs = []
    val_MAEs = []

    @tf.function
    def train_step(x, y):
        with tf.GradientTape() as tape:
            decoder_preds = autoencoder(x, training=True)
            if reconstruct_with_HFENN:

                loss_value_hfenn = hfenn(x, decoder_preds)
                loss_value_reconstruction = reconstruction_loss(x, decoder_preds)
                loss = (lambdaHFENN * loss_value_hfenn) + (lambdaR * loss_value_reconstruction)
            else:
                loss = lambdaR * reconstruction_loss(x, decoder_preds)

        grads = tape.gradient(loss, autoencoder.trainable_weights)
        optimizer.apply_gradients(zip(grads, autoencoder.trainable_weights))
        train_MAE_metric.update_state(x, decoder_preds)
        return loss

    @tf.function
    def test_step(x, y):
        val_preds = autoencoder(x, training=False)
        val_MAE_metric.update_state(x, val_preds)

    epoch = 0
    epochs_wo_improving = 0
    reconstruction_convergence = 0

    while reconstruction_convergence == 0:
        print(f"\nEpoch {epoch}")
        for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
            loss_value = train_step(x_batch_train, y_batch_train)
            if step % 100 == 0:
                print(f"Step {step}: loss = {float(loss_value):.4f}")

        train_MAE = train_MAE_metric.result()
        val_MAE = 0.0
        train_MAE_metric.reset_state()

        for x_val, y_val in test_dataset:
            test_step(x_val, y_val)

        val_MAE = val_MAE_metric.result()
        val_MAE_metric.reset_state()

        print(f"Train MAE: {train_MAE:.4f} | Val MAE: {val_MAE:.4f}")

        train_losses.append(float(loss_value))
        train_MAEs.append(float(train_MAE))
        val_MAEs.append(float(val_MAE))

        if epoch == 0:
            best_val_MAE = val_MAE
        elif val_MAE < best_val_MAE:
            best_val_MAE = val_MAE
            epochs_wo_improving = 0
            encoder.save(store_encoder_path)
            decoder.save(store_decoder_path)
            autoencoder.save(store_AE_path)
        else:
            epochs_wo_improving += 1

        if reconstruction_patience_epochs < epochs_wo_improving and epoch != 0:
            reconstruction_convergence = 1

        epoch += 1

    print(f"✅ Finished training for Speaker {speaker} in {epoch} epochs.")

## **Evaluation**

##### Functions

In [ ]:
def mel_to_audio(mel_spec):
    mel_spec = librosa.db_to_power(mel_spec)
    audio = librosa.feature.inverse.mel_to_audio(mel_spec, sr=sr,
                                                 hop_length=hop_length,
                                                 window=window,
                                                 win_length=win_length,
                                                 n_fft=n_fft,
                                                 )
    return audio

In [ ]:
def load_models(speaker):
    stored_autoencoder_path = f'/home/jovyan/Daniel/Exp_{dataset}/autoencoders_training/models/autoencoder_Speaker{speaker}.keras'
    stored_encoder_path = f'/home/jovyan/Daniel/Exp_{dataset}/autoencoders_training/models/encoder_Speaker{speaker}.keras'
    stored_decoder_path = f'/home/jovyan/Daniel/Exp_{dataset}/autoencoders_training/models/decoder_Speaker{speaker}.keras'

    autoencoder = keras.models.load_model(stored_autoencoder_path)
    encoder = keras.models.load_model(stored_encoder_path)
    decoder = keras.models.load_model(stored_decoder_path)

    return autoencoder, encoder, decoder

In [ ]:
def displayReconstructionVertical(speaker, plt_name, n, x_to_show, decoder_preds, n_frames_cnn, n_freq_cnn, displayEdges=False, display_sobel=False, LoG=False):
    size = 'big'
    fontsize = 48
    if size == 'small':
        size_x = (n_freq_cnn / 3) / 10
        size_y = n_frames_cnn / 10
    elif size == 'big':
        size_x = (n_freq_cnn * 6) / 2
        size_y = n_frames_cnn / 2

    num_imgs_per_row = n
    columns = n // int(num_imgs_per_row/2)
    show = 'img'

    if not displayEdges:
      scale = 'Hz'
    else:
      scale = 'Mel'
    colorbar = False

    num_show = 0

    for i in range(columns):
        fig = plt.figure(figsize=(size_x, size_y))

        for j in range(num_imgs_per_row):
            ax = plt.subplot(2, num_imgs_per_row, j + 1)  # Change to 2 rows per figure

            if show == 'img':
                image = x_to_show[num_show]
                title = r"$X_{{{}}}$".format(num_show)
            if show == 'recons':
                if displayEdges:
                  if display_sobel:
                    x = K.expand_dims(x_to_show,-1)
                    x_tensor = tf.convert_to_tensor(x)
                    sobel = tf.image.sobel_edges(x_tensor)
                    sobel_x = sobel[..., 0]  # Horizontal edges
                    sobel_y = sobel[..., 1]  # Vertical edges
                    image = tf.sqrt(tf.square(sobel_x[num_show]) + tf.square(sobel_y[num_show]))
                    title = r"$G(X_{{{}}})$".format(num_show)
                  if LoG:
                    x = x_to_show[num_show].reshape(n_frames_cnn, n_freq_cnn)
                    blurred = cv2.GaussianBlur(x, (5, 5), 0)  # Apply Gaussian Blur
                    image = cv2.Laplacian(blurred, cv2.CV_64F)  # Apply Laplacian filter
                    title = r"""$LoG(X_{})$""".format(num_show)
                else:
                  image = decoder_preds[num_show]
                  title = r"$\tilde{{X}}_{{{}}}$".format(num_show)
                num_show += 1

            if scale == 'Mel':
                if show=='recons':
                  im = plt.imshow(image, origin='lower')
                else:
                  im = plt.imshow(image.reshape(n_frames_cnn, n_freq_cnn), origin='lower')

                plt.title(title, fontsize=fontsize)

                if colorbar or j==(num_imgs_per_row-1):
                    divider = make_axes_locatable(ax)
                    cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
                    plt.colorbar(im, cax=cax, format="%+2.f dB")

            if scale == 'Hz':
                im = librosa.display.specshow(image, cmap='viridis', x_axis='time', y_axis='mel', sr=48000)

                plt.title(title, fontsize=fontsize)

                if colorbar or j==(num_imgs_per_row-1):
                    divider = make_axes_locatable(ax)
                    cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
                    plt.colorbar(im, cax=cax, format="%+2.f dB")

            plt.xticks(fontsize=fontsize)
            plt.yticks(fontsize=fontsize)
            ax.get_xaxis().set_ticks([])
            ax.set_xlabel('Time', fontsize=fontsize)
            if displayEdges:
              ax.set_ylabel('Mel', fontsize=fontsize)
            else:
              ax.set_ylabel('Hz', fontsize=fontsize)
            if j > 0:
              ax.get_yaxis().set_visible(False)

            if show == 'img':
              show = 'recons'
            else:
              show ='img'

        plt.savefig(f'/home/jovyan/Daniel/Exp_{dataset}/autoencoders_training/evaluation/Spe{speaker}_{plt_name}_{i}.png')
        plt.show()
        plt.close()

##### Loop

In [ ]:
results = []

#for speaker_id in range(1, n_speakers + 1):
for speaker in speakers:
    speaker = f"{speaker_id:02d}"  # format as '01', '02', ..., '50'
    print(f"\n\n🔁 Evaluating Autoencoder for Speaker {speaker}...")

    # Define paths
    data_path = f'/home/jovyan/Daniel/Exp_{dataset}/datasets/preprocessed/{dataset}100ms32LabeledSpe{speaker}.npz'

    # Load data
    data = np.load(data_path)
    x_train_100ms = data['x_train']
    y_train_100ms = data['y_emo']

    x_train, x_test, y_train, y_test = train_test_split(
        x_train_100ms, y_train_100ms, test_size=test_prctg, random_state=42)

    # Normalization
    min_norm = np.min(x_train)
    max_norm = np.max(x_train)

    x_train_norm = (x_train - min_norm) / (max_norm - min_norm)
    x_test_norm = (x_test - min_norm) / (max_norm - min_norm)

    # Load models
    autoencoder, encoder, decoder = load_models(speaker)

    autoencoder_mae_metric = keras.metrics.MeanAbsoluteError()
    autoencoder.compile(optimizer='adam', loss='mean_squared_error', metrics=[autoencoder_mae_metric])
    autoencoder_mae = autoencoder.evaluate(x_test_norm, x_test_norm, verbose=0)
    mae_value = autoencoder_mae[1]
    print(f"Autoencoder MAE: {mae_value}")

    # Store the result
    results.append({'Speaker': speaker, 'MAE': mae_value})

    # Qualitative assessment
    n=12
    x_to_show = x_test_norm[:n]
    decoder_preds = autoencoder.predict(x_to_show)
    reconstructions_rescaled = decoder_preds * (max_norm - min_norm) + min_norm

    displayReconstructionVertical(speaker=speaker,
                                  plt_name='reconstruction',
                                  n=n,
                                  x_to_show=x_to_show,
                                  decoder_preds=reconstructions_rescaled,
                                  n_frames_cnn=x_train.shape[1],
                                  n_freq_cnn=x_train.shape[2])

    # Gradient loss display
    n=12
    x_to_show = x_train[:n]
    decoder_preds = autoencoder.predict(x_to_show)

    displayReconstructionVertical(speaker=speaker,
                                  plt_name='sobel_edges',
                                  n=n,
                                  x_to_show=x_to_show,
                                  decoder_preds=reconstructions_rescaled,
                                  n_frames_cnn=x_train.shape[1],
                                  n_freq_cnn=x_train.shape[2],
                                  displayEdges = True,
                                  display_sobel=True)

    # Audio reconstruction
    x_test_reconstructed = autoencoder.predict(x_test_norm)

    for i, mel_spec in enumerate(x_test[:5]):
        audio = mel_to_audio(mel_spec)
        filename = f'audio_{i}.wav'
        wavfile.write(filename, sr, audio)
        display(Audio(filename))

    reconstructions_rescaled = x_test_reconstructed * (max_norm - min_norm) + min_norm
    print('Audio reconstruction through Inverse Fourier Transform:')

    for i, mel_spec in enumerate(reconstructions_rescaled[:5]):
        audio = mel_to_audio(mel_spec)
        filename = f'recons_audio_{i}.wav'
        wavfile.write(filename, sr, audio)
        display(Audio(filename))

    print(f"✅ Finished evaluating Speaker {speaker}.")

In [ ]:
results_df = pd.DataFrame(results)
results_csv_path = f'/home/jovyan/Daniel/Exp_{dataset}/autoencoders_training/evaluation/mae_results.csv'
results_df.to_csv(results_csv_path, index=False)
print(f"Saved MAE results table to {results_csv_path}")

# Plot the table as an image
fig, ax = plt.subplots(figsize=(8, len(results_df) * 0.3 + 1))
ax.axis('off')
table = ax.table(cellText=results_df.values,
                 colLabels=results_df.columns,
                 cellLoc='center',
                 loc='center')
table.scale(1, 1.5)

# Save the image
image_path = f'/home/jovyan/Daniel/Exp_{dataset}/autoencoders_training/evaluation/mae_results_table.png'
plt.savefig(image_path, bbox_inches='tight')
plt.close()

print(f"Saved MAE table image to {image_path}")